In [6]:
import pandas as pd
import os

def clean_feature_file(features_filepath, invalid_list_filepath, output_filepath, id_column_name):
    """
    Removes rows from a feature file by matching the last number of an identifier
    against the values in the specified ID column.

    Args:
        features_filepath (str): Path to the main CSV feature file.
        invalid_list_filepath (str): Path to the .txt file with invalid IDs.
        output_filepath (str): Path to save the new, cleaned CSV file.
        id_column_name (str): The name of the column in the CSV that contains the turn ID numbers.
    """
    print("Starting the data cleaning process with the new matching logic...")

    # 1. Load the features CSV file
    try:
        features_df = pd.read_csv(features_filepath)
        print(f"✅ Successfully loaded feature file: {os.path.basename(features_filepath)}")
        print(f"   Original number of rows: {len(features_df)}")
    except FileNotFoundError:
        print(f"❌ Error: The feature file was not found at '{features_filepath}'")
        return

    # 2. Load the invalid identifiers and extract ONLY THE LAST NUMBER
    invalid_ids = set()
    try:
        with open(invalid_list_filepath, 'r') as f:
            for line in f:
                if line.strip():
                    # Split the line by '_' and take the last part, then convert to an integer
                    try:
                        turn_id_to_remove = int(line.strip().split('_')[-1])
                        invalid_ids.add(turn_id_to_remove)
                    except (ValueError, IndexError):
                        print(f"  - Warning: Could not parse ID from line: '{line.strip()}'")

        print(f"✅ Successfully extracted {len(invalid_ids)} invalid Turn IDs to remove.")
    except FileNotFoundError:
        print(f"❌ Error: The invalid list was not found at '{invalid_list_filepath}'")
        return

    # 3. Check if the identifier column exists
    if id_column_name not in features_df.columns:
        print(f"❌ Error: Column '{id_column_name}' not found in the CSV file.")
        print(f"   Available columns are: {list(features_df.columns)}")
        return

    # 4. Filter the DataFrame using the set of invalid numbers
    original_rows = len(features_df)
    cleaned_df = features_df[~features_df[id_column_name].isin(invalid_ids)]
    rows_removed = original_rows - len(cleaned_df)

    print(f"\nFiltering complete.")
    print(f"   {rows_removed} rows were removed.")
    print(f"   Number of rows after cleaning: {len(cleaned_df)}")

    # 5. Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_filepath, index=False)
    print(f"✅ Successfully saved cleaned data to: {output_filepath}")


# --- SCRIPT CONFIGURATION ---
identifier_column = 'Turn ID'

# --- FILE PATHS ---
features_file = r'D:\Users\Kaiyang\OneDrive\The University of Colorado Denver\OneDrive - The University of Colorado Denver\Courses\thesis\data_exploration\chat_gpt_test\turning_project_2d_features.csv'
invalid_ids_file = r'D:\Users\Kaiyang\OneDrive\The University of Colorado Denver\OneDrive - The University of Colorado Denver\Thesis\Code\DataExploration\invalid_data_list.txt'
cleaned_output_file = r'D:\Users\Kaiyang\OneDrive\The University of Colorado Denver\OneDrive - The University of Colorado Denver\Courses\thesis\data_exploration\chat_gpt_test\cleaned_turning_project_2d_features.csv'


# --- RUN THE SCRIPT ---
if __name__ == "__main__":
    clean_feature_file(
        features_filepath=features_file,
        invalid_list_filepath=invalid_ids_file,
        output_filepath=cleaned_output_file,
        id_column_name=identifier_column
    )

Starting the data cleaning process with the new matching logic...
✅ Successfully loaded feature file: turning_project_2d_features.csv
   Original number of rows: 1749
✅ Successfully extracted 68 invalid Turn IDs to remove.

Filtering complete.
   68 rows were removed.
   Number of rows after cleaning: 1681
✅ Successfully saved cleaned data to: D:\Users\Kaiyang\OneDrive\The University of Colorado Denver\OneDrive - The University of Colorado Denver\Courses\thesis\data_exploration\chat_gpt_test\cleaned_turning_project_2d_features.csv


In [1]:
import pandas as pd
import re
import os

# --- Define the file names ---
sits_labels_path = 'SitToStand_human_labels.csv'
invalid_list_path = 'invalid_data_list.txt'
output_csv_path = 'invalid_participants_data.csv'

def filter_by_invalid_participants(sits_path, invalid_path, output_path):
    """
    Filters a CSV to keep only rows corresponding to participant IDs
    found in an invalid list.
    """
    print("--- Starting Filtering Process ---")

    # 1. Load the source data CSV
    try:
        source_df = pd.read_csv(sits_path)
        print(f"✅ Successfully loaded source data: '{os.path.basename(sits_path)}'")
    except FileNotFoundError:
        print(f"❌ Error: The file was not found. Please make sure '{sits_path}' is in the same folder as the script.")
        return
    except Exception as e:
        print(f"An error occurred while processing '{sits_path}': {e}")
        return

    # 2. Load the invalid list and extract the Participant ID number
    invalid_participant_ids = set()
    try:
        with open(invalid_path, 'r') as f:
            for line in f:
                if line.strip():
                    # Use a regular expression to find the number immediately after "Pt"
                    match = re.search(r'Pt(\d+)', line, re.IGNORECASE)
                    if match:
                        participant_id = int(match.group(1))
                        invalid_participant_ids.add(participant_id)

        print(f"✅ Successfully extracted {len(invalid_participant_ids)} unique Participant IDs from '{os.path.basename(invalid_path)}'.")

    except FileNotFoundError:
        print(f"❌ Error: The file was not found. Please make sure '{invalid_path}' is in the same folder as the script.")
        return
    except Exception as e:
        print(f"An error occurred while processing '{invalid_path}': {e}")
        return

    # 3. Define the column to match on and filter the DataFrame
    id_column_name = 'Participant ID number'
    if id_column_name not in source_df.columns:
        print(f"❌ Error: Column '{id_column_name}' not found in the source CSV.")
        return

    # Keep only the rows where the 'Participant ID number' is in our invalid set
    filtered_df = source_df[source_df[id_column_name].isin(invalid_participant_ids)]

    # 4. Save the new CSV file
    filtered_df.to_csv(output_path, index=False)
    print(f"\n--- Results ---")
    print(f"✅ Successfully saved the filtered data for {len(filtered_df)} rows to '{output_path}'.")


# --- RUN THE SCRIPT ---
if __name__ == "__main__":
    filter_by_invalid_participants(
        sits_path=sits_labels_path,
        invalid_path=invalid_list_path,
        output_path=output_csv_path
    )

--- Starting Filtering Process ---
✅ Successfully loaded source data: 'SitToStand_human_labels.csv'
✅ Successfully extracted 20 unique Participant IDs from 'invalid_data_list.txt'.

--- Results ---
✅ Successfully saved the filtered data for 329 rows to 'invalid_participants_data.csv'.
